# Real Estate Data Processing - Combining Buyer and Transaction Data

## Overview
This notebook demonstrates how to join real estate sales data from multiple CSV files and convert the result to JSON objects. The main focus is on joining buyer information with their corresponding transaction data.



In [1]:
# Real Estate Data Processing Notebook
# This notebook joins buyers data with call transcripts and converts to JSON

import pandas as pd
import json
import os
from datetime import datetime

# Function to load our CSV files
def load_data(file_path):
    """
    Load a CSV file into a pandas DataFrame
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {file_path} with {len(df)} records")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Load all CSV files
buyers_df = load_data("real_estate_synthetic_data/buyers.csv")
houses_df = load_data("real_estate_synthetic_data/houses_for_sale.csv")
past_sales_df = load_data("real_estate_synthetic_data/past_sales.csv")
transcripts_df = load_data("real_estate_synthetic_data/basic_call_transcripts.csv")


# Display the first few rows of each DataFrame to verify data was loaded correctly
print("\nBuyers Data Sample:")
print(buyers_df.head(2))

print("\nHouses Data Sample:")
print(houses_df.head(2))

print("\nPast Sales Data Sample:")
print(past_sales_df.head(2))

print("\nTranscripts Data Sample:")
print(transcripts_df.head(2))

# Check for null values in BuyerID field for both DataFrames
print(f"\nNull BuyerIDs in buyers_df: {buyers_df['BuyerID'].isnull().sum()}")
print(f"Null BuyerIDs in transcripts_df: {transcripts_df['BuyerID'].isnull().sum()}")


Successfully loaded real_estate_synthetic_data/buyers.csv with 5000 records
Successfully loaded real_estate_synthetic_data/houses_for_sale.csv with 1000 records
Successfully loaded real_estate_synthetic_data/past_sales.csv with 500 records
Successfully loaded real_estate_synthetic_data/basic_call_transcripts.csv with 10000 records

Buyers Data Sample:
                                BuyerID         FullName  \
0  84092373-60de-427b-a0a7-a9f096e107a3   Nathan Sanchez   
1  42b70d65-9cfd-481c-9b12-4dc862dab833  Cindy Henderson   

                                             Address          SSN  \
0  4009 Carter Alley Suite 217, West Windsor, NJ ...  702-78-5163   
1    635 Steven Meadows, Franklin Township, NJ 08498  455-07-1143   

            PhoneNumber                        Email IncomeBracket  \
0   (307)773-5111x30556      owashington@example.com        Medium   
1  001-926-910-8836x670  adrienneedwards@example.com           Low   

   AnnualIncome  NetWorth  TotalDebt  CreditSc

In [ ]:
# Join buyers and transcripts data on BuyerID
merged_df = pd.merge(
    buyers_df, 
    transcripts_df, 
    on='BuyerID', 
    how='left',  # Keep all buyers, even those without transcripts
    suffixes=('', '_transcript')
)

print(f"\nMerged data shape: {merged_df.shape}")
print("\nMerged Data Sample:")
print(merged_df.head(2))

# Count how many buyers have call transcripts
buyers_with_transcripts = merged_df.dropna(subset=['TranscriptID']).shape[0]
print(f"\nBuyers with call transcripts: {buyers_with_transcripts} out of {len(buyers_df)} total buyers")

# Optional: Organize and clean the data
# For example, convert date strings to datetime objects
if 'CallDateTime' in merged_df.columns:
    merged_df['CallDateTime'] = pd.to_datetime(merged_df['CallDateTime'], errors='coerce')

# Create a function to convert the DataFrame to a list of JSON objects
def dataframe_to_json(df):
    """
    Convert a DataFrame to a list of JSON objects (dictionaries)
    """
    # Convert to dict records
    json_data = df.to_dict(orient='records')
    
    # Optional: Clean up the JSON data
    for record in json_data:
        # Remove NaN values
        for key, value in list(record.items()):
            if pd.isna(value):
                record[key] = None
    
    return json_data

# Create the json_output directory if it doesn't exist
json_output_dir = "json_output"
os.makedirs(json_output_dir, exist_ok=True)
print(f"\nCreated directory: {json_output_dir}")

# Convert the merged data to JSON
json_data = dataframe_to_json(merged_df)

# Print the first JSON object as a sample
print("\nSample JSON object:")
print(json.dumps(json_data[0], indent=2, default=str))

# Now let's save the JSON data to a file in the json_output directory
json_file_path = os.path.join(json_output_dir, 'buyers_with_transcripts.json')
with open(json_file_path, 'w') as f:
    json.dump(json_data, f, indent=2, default=str)

print(f"\nSaved {len(json_data)} records to {json_file_path}")



Merged data shape: (10666, 19)

Merged Data Sample:
                                BuyerID         FullName                                              Address          SSN           PhoneNumber                        Email IncomeBracket  AnnualIncome  NetWorth  TotalDebt  CreditScore  DesiredPriceRange_Min  DesiredPriceRange_Max  PreApprovedAmount                          TranscriptID                CallDateTime        BrokerName MortgageBankerName                                                                                                                                                                                                                                                                                                                                    TranscriptText
0  84092373-60de-427b-a0a7-a9f096e107a3   Nathan Sanchez  4009 Carter Alley Suite 217, West Windsor, NJ 08618  702-78-5163   (307)773-5111x30556      owashington@example.com        Medium        117249    

In [ ]:

# Optional: Create a more complex join with houses and past sales
# This creates a comprehensive view of buyers, their transcripts, and any properties they purchased

# First join past sales with buyers
buyers_sales_df = pd.merge(
    buyers_df,
    past_sales_df,
    on='BuyerID',
    how='left',
    suffixes=('', '_sale')
)

# Then join with transcripts
complete_df = pd.merge(
    buyers_sales_df,
    transcripts_df,
    on='BuyerID',
    how='left',
    suffixes=('', '_transcript')
)

# Convert to JSON and save
complete_json = dataframe_to_json(complete_df)

# Save the file in the json_output directory
json_file_path = os.path.join(json_output_dir, 'complete_real_estate_data.json')
with open(json_file_path, 'w') as f:
    json.dump(complete_json, f, indent=2, default=str)

print(f"\nSaved {len(complete_json)} records to {json_file_path}")

# Generate a summary of the data
print("\nData Summary:")
print(f"Total buyers: {len(buyers_df)}")
print(f"Total houses for sale: {len(houses_df)}")
print(f"Total past sales: {len(past_sales_df)}")
print(f"Total call transcripts: {len(transcripts_df)}")



Saved 10666 records to complete_real_estate_data.json

Data Summary:
Total buyers: 5000
Total houses for sale: 1000
Total past sales: 500
Total call transcripts: 10000


In [ ]:
# Create a more tailored JSON structure
# This is a custom structure that might better represent the relationships
def create_custom_json_structure(buyers, transcripts, sales):
    """
    Create a custom nested JSON structure with buyers as the top level,
    and their transcripts and purchases as nested objects
    """
    # Create a dictionary to store all data by BuyerID
    buyer_data = {}
    
    # Process all buyers
    for _, buyer in buyers.iterrows():
        buyer_id = buyer['BuyerID']
        buyer_data[buyer_id] = {
            'buyer_info': buyer.to_dict(),
            'transcripts': [],
            'purchases': []
        }
    
    # Add transcripts to their respective buyers
    for _, transcript in transcripts.iterrows():
        buyer_id = transcript['BuyerID']
        if buyer_id in buyer_data:
            buyer_data[buyer_id]['transcripts'].append(transcript.to_dict())
    
    # Add purchases to their respective buyers
    for _, sale in sales.iterrows():
        buyer_id = sale['BuyerID']
        if buyer_id in buyer_data:
            buyer_data[buyer_id]['purchases'].append(sale.to_dict())
    
    # Convert to a list of objects
    result = list(buyer_data.values())
    
    return result

# Create the custom structure
custom_json = create_custom_json_structure(buyers_df, transcripts_df, past_sales_df)

# Save the custom JSON structure to the json_output directory
custom_json_path = os.path.join(json_output_dir, 'custom_real_estate_data.json')
with open(custom_json_path, 'w') as f:
    json.dump(custom_json, f, indent=2, default=str)

print(f"\nSaved {len(custom_json)} records to {custom_json_path}")

# Example of how to load the JSON data back
def load_json_data(file_path):
    """
    Load JSON data from a file
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        print(f"Successfully loaded {len(data)} records from {file_path}")
        return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Test loading the created JSON files
# Uncomment these lines to test
# loaded_json = load_json_data(os.path.join(json_output_dir, 'custom_real_estate_data.json'))
# print(f"First record: {json.dumps(loaded_json[0], indent=2)[:500]}...")


Saved 5000 records to custom_real_estate_data.json
